In [1]:
import Control.Monad (void)
import Control.Monad.Identity (Identity)
import Control.Monad.Trans.State
import Control.Monad.IO.Class (liftIO)
import Control.Applicative (liftA2)
import System.Random (getStdRandom, randomR)
import Data.Function (on)

In [6]:
data Player = Odd | Even deriving Show

data Game = Game { oddScore :: Int, evenScore :: Int, rounds :: [Round] }

instance Show Game where
    show g = "Game {oddScore = " ++ show (oddScore g) ++ ", evenScore = " ++ show (evenScore g) ++ "}"

type Winner = Player
data GameState = InProgress Game | Finished Winner deriving Show

data Move = One | Two deriving Show

data Round = Round { oddMove :: Move, evenMove :: Move } deriving Show

initialGame :: Game
initialGame = Game 0 0 []

moveToInt :: Move -> Int
moveToInt One = 1
moveToInt Two = 2

intToMove :: Int -> Move
intToMove 1 = One
intToMove 2 = Two

applyRound :: Round -> Game -> Game
applyRound r g = 
    if even total 
    then g { evenScore = evenScore g + 1, rounds = r:rounds g} 
    else g { oddScore = oddScore g + 1, rounds = r:rounds g} 
    where
        total = liftA2 ((+) `on` moveToInt) oddMove evenMove r

getGameState :: Game -> GameState
getGameState g
    | oddScore g >= 3 = Finished Odd
    | evenScore g >= 3 = Finished Even
    | otherwise = InProgress g

playRound :: Round -> Game -> GameState
playRound r = getGameState . applyRound r

doRound :: StateT Game IO Round
doRound = do
    m1 <- intToMove <$> liftIO readLn
    m2 <- intToMove <$> liftIO (getStdRandom (randomR (1,2)))
    return $ Round m1 m2  

doGame :: IO ()
doGame = evalStateT (liftIO (putStrLn "You play as odd.") >> loop) initialGame where
    printGame :: StateT Game IO ()
    printGame = get >>= liftIO . print

    loop :: StateT Game IO ()
    loop = do        
        r <- doRound
        liftIO $ print r
        g <- get
        case playRound r g of
            InProgress g' -> do
                put g'
                printGame
                loop
            Finished winner -> liftIO $ putStrLn ("winner: " ++ show winner)
            
doGame

You play as odd.
Round {oddMove = One, evenMove = Two}
Game {oddScore = 1, evenScore = 0}
Round {oddMove = Two, evenMove = One}
Game {oddScore = 2, evenScore = 0}
Round {oddMove = One, evenMove = Two}
winner: Odd